Hardcode an array saving images and its prediction values

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.python.keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Nadam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
from tensorflow.python.keras.models import Model
from sklearn.utils import shuffle
import savetonparray as savetonp

# import loaddata as loaddata
from google.colab import drive
# %matplotlib inline

# drive.mount('/content/drive')
# !rm -rf '/content/train'
# savetonp.run('train')

In [ ]:
train_x = np.load('train.npy')
test_x = np.load('test.npy')
# train_x = loaddata.run('train')
# test_x = loaddata.run('test')
train_y = pd.read_csv('train_result.csv').iloc[:,1:].values
test_y = pd.read_csv('test_result.csv').iloc[:,1:].values

train_x = train_x/255.
test_x = test_x/255.
train_y = train_y/100.
test_y = test_y/100.

train_x, train_y = shuffle(train_x, train_y)
test_x, test_y = shuffle(test_x, test_y)
# shuffle train labels and samples with respect to each other

print ("train_x shape: " + str(train_x.shape))
print ("train_y shape: " + str(train_y.shape))
print ("test_x shape: " + str(test_x.shape))
print ("test_y shape: " + str(test_y.shape))
print(test_y)

# shuffle*, and add 5 more y result to csv

train_x shape: (335, 299, 299, 15)
train_y shape: (335, 3)
test_x shape: (90, 299, 299, 15)
test_y shape: (90, 3)
[[0.28 0.24 0.48]
 [0.   0.04 0.96]
 [0.62 0.16 0.22]
 [0.61 0.27 0.12]
 [0.43 0.22 0.35]
 [0.28 0.3  0.42]
 [0.52 0.28 0.2 ]
 [0.3  0.31 0.39]
 [0.4  0.27 0.33]
 [0.   0.04 0.96]
 [0.74 0.16 0.1 ]
 [0.22 0.49 0.29]
 [0.   0.04 0.96]
 [0.61 0.27 0.12]
 [0.38 0.33 0.29]
 [0.51 0.27 0.22]
 [0.5  0.35 0.15]
 [0.22 0.49 0.29]
 [0.74 0.16 0.1 ]
 [0.3  0.31 0.39]
 [0.43 0.22 0.35]
 [0.51 0.27 0.22]
 [0.4  0.27 0.33]
 [0.28 0.3  0.42]
 [0.61 0.27 0.12]
 [0.4  0.27 0.33]
 [0.19 0.25 0.56]
 [0.62 0.16 0.22]
 [0.18 0.34 0.48]
 [0.28 0.24 0.48]
 [0.4  0.27 0.33]
 [0.62 0.16 0.22]
 [0.5  0.35 0.15]
 [0.   0.04 0.96]
 [0.38 0.33 0.29]
 [0.19 0.25 0.56]
 [0.28 0.24 0.48]
 [0.5  0.35 0.15]
 [0.22 0.49 0.29]
 [0.28 0.3  0.42]
 [0.66 0.24 0.1 ]
 [0.28 0.3  0.42]
 [0.74 0.16 0.1 ]
 [0.3  0.31 0.39]
 [0.   0.04 0.96]
 [0.22 0.49 0.29]
 [0.28 0.3  0.42]
 [0.68 0.19 0.13]
 [0.68 0.19 0.13]
 [0.

In [ ]:
# build our classifier model based on pre-trained InceptionResNetV2:
# 1. we don't include the top (fully connected) layers of InceptionResNetV2
# 2. we add a DropOut layer followed by a Dense (fully connected)
#    layer which generates softmax class score for each class
# 3. we compile the final model using an Adam optimizer, with a
#    low learning rate (since we are 'fine-tuning')
model = InceptionResNetV2(include_top=True,
                      weights=None,
                      input_shape=(299,299,15),
                      classes=3)

model.compile(optimizer=Nadam(learning_rate=0.002, beta_1=0.9, beta_2=0.999, name="Nadam",),
                  loss='MSE', metrics=['MAE'])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/applications/imagenet_utils.py:333: UserWarning: This model usually expects 1 or 3 input channels. However, it was passed an input_shape with 15 input channels.
  str(input_shape[-1]) + ' input channels.')


In [ ]:
print(model.summary())

Model: "inception_resnet_v2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 299, 299, 15 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 149, 149, 32) 4320        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 149, 149, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 149, 149, 32) 0           batch_normalization[0][0]        
________________________________________________________________________________

In [5]:
# train the model
model.fit(train_x, train_y, epochs = 20, batch_size = 15)


Epoch 1/20
23/23 [==============================] - 705s 28s/step - loss: 0.1510 - MAE: 0.3232
Epoch 2/20
23/23 [==============================] - 647s 28s/step - loss: 0.1136 - MAE: 0.2849
Epoch 3/20
23/23 [==============================] - 647s 28s/step - loss: 0.0451 - MAE: 0.1699
Epoch 4/20
23/23 [==============================] - 646s 28s/step - loss: 0.0255 - MAE: 0.1150
Epoch 5/20
23/23 [==============================] - 647s 28s/step - loss: 0.0352 - MAE: 0.1320
Epoch 6/20
23/23 [==============================] - 647s 28s/step - loss: 0.0424 - MAE: 0.1405
Epoch 7/20
23/23 [==============================] - 647s 28s/step - loss: 0.0465 - MAE: 0.1503
Epoch 8/20
23/23 [==============================] - 651s 28s/step - loss: 0.0295 - MAE: 0.1250
Epoch 9/20
23/23 [==============================] - 650s 28s/step - loss: 0.0246 - MAE: 0.1205
Epoch 10/20
23/23 [==============================] - 648s 28s/step - loss: 0.0484 - MAE: 0.1526
Epoch 11/20
23/23 [==============================

In [6]:
# save trained weights
# model.save('15stacked_InceptioResnetV2.h5')
model.save_weights('InceptionResNetV2_10epochs_15batches_Nadam_MSE_MAE.h5')
# predictions = net_final.predict(x=test_batches, verbose=0)

In [7]:
preds = model.evaluate(test_x, test_y)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))
print(preds)

3/3 [==============================] - 45s 13s/step - loss: 0.1083 - MAE: 0.2743
Loss = 0.1082962155342102
Test Accuracy = 0.2742944061756134
[0.1082962155342102, 0.2742944061756134]


In [ ]:
# x_pred1 = np.load('test_img_9.npy')
# print(x_pred1.shape)
# # # img_path = 'resize_5_2.png'
# # img = image.load_img(x, target_size=(224, 224))
# # x = image.img_to_array(img)
# # x = np.expand_dims(x, axis=0)
# x_pred1 = x_pred1/255.0
# print('Input image shape:', x_pred1.shape)
# print("class prediction vector [PR(0), N-PR(1), IM(2)] = ")
# print((net_final.predict(x_pred1)*100).astype(int))
# print(test_y[8]*100)

In [8]:
y_pred_array = np.empty(((test_y.shape[0]), 3), dtype=int)
for i in range (test_x.shape[0]):
  x_pred = test_x[i,:,:,:]
  x_pred = x_pred.reshape((1,299,299,15))
  # print(x_pred.shape)
  y_pred = model.predict(x_pred)
  y_pred_array[i] = (y_pred*100).astype(int)

In [9]:
print(y_pred_array)
print(test_y*100)
np.savetxt('predicted_y_20epochs15batch.csv', y_pred_array, fmt='%5.0f', delimiter=",")
np.savetxt('groundtruth_y_20epochs15batch.csv.csv', test_y*100, fmt='%5.0f', delimiter=",")

[[ 17  14  67]
 [ 16  14  68]
 [ 17  14  68]
 [ 19  15  64]
 [ 18  14  66]
 [ 17  14  67]
 [  0   0 100]
 [ 17  14  67]
 [ 17  14  67]
 [ 16  14  68]
 [ 17  14  68]
 [ 17  14  68]
 [ 16  14  68]
 [ 19  16  64]
 [ 19  16  64]
 [ 17  15  66]
 [ 23  14  61]
 [ 17  14  68]
 [ 17  14  68]
 [ 17  14  67]
 [ 18  14  66]
 [ 17  14  67]
 [ 18  14  67]
 [ 17  14  67]
 [ 19  15  64]
 [ 17  14  67]
 [ 17  14  67]
 [ 17  14  68]
 [  0   0 100]
 [ 17  14  67]
 [ 17  14  67]
 [ 17  14  68]
 [ 24  15  59]
 [ 16  14  68]
 [ 19  15  64]
 [ 17  14  67]
 [ 17  14  67]
 [ 25  15  58]
 [ 17  14  68]
 [ 17  14  67]
 [ 22  14  63]
 [ 17  14  67]
 [ 17  14  68]
 [ 18  14  67]
 [ 16  14  68]
 [ 17  14  68]
 [ 17  14  67]
 [ 16  14  68]
 [ 16  14  68]
 [ 17  14  67]
 [ 17  14  67]
 [ 17  14  67]
 [  0   0 100]
 [  0   0 100]
 [ 17  14  68]
 [ 17  14  67]
 [ 23  14  61]
 [ 25  16  58]
 [ 18  14  66]
 [ 17  14  67]
 [ 25  16  57]
 [  0   0 100]
 [ 17  14  67]
 [ 17  14  68]
 [ 19  16  64]
 [ 18  14  66]
 [  0   0 